# 1.데이터 생성

## 1.1 고객번호, 연령, 소득구간 할당

In [ ]:
import pandas as pd
#1~10000까지의 고객 ID를 부여 후 DataFrame 생성

df = pd.DataFrame(range(1, 10001), columns = ['고객ID'])
df = df.set_index('고객ID')

df

In [ ]:
import numpy as np

# 20~29까지 각 1000개씩 연령 column생성 후 랜덤하게 섞어줌

np.random.seed(42)
ages = np.repeat(range(20, 30), 1000)
np.random.shuffle(ages)
df['연령'] = ages

In [ ]:
# 소득 구간(1~5분위) 열 생성 

np.random.seed(42)
income_groups = {1: 94, 2: 191, 3: 216, 4: 268, 5: 458}

# 나이에 따라 소득구간의 분포를 다르게 배열
age_group_ratios = {
    (20, 21): [0.2, 0.4, 0.2, 0.1, 0.1],
    (22, 23): [0.1, 0.3, 0.3, 0.2, 0.1],
    (24, 25): [0.1, 0.2, 0.3, 0.3, 0.1],
    (26, 27): [0.1, 0.2, 0.2, 0.3, 0.2],
    (28, 29): [0.1, 0.1, 0.2, 0.3, 0.3]
}
for i in range(1, 1+len(df)):
    age = df.loc[i, '연령']
    for age_group, ratios in age_group_ratios.items():
        if age in range(age_group[0], age_group[1]+1):
            income = np.random.uniform(0.9, 1.1, size=5) * [income_groups[i] for i in range(1, 6)]
            df.loc[i, '소득구간'] = np.random.choice(range(1, 6), p=ratios)
            break

In [ ]:
# 각 소득분위에 대한 평균 소득 금액 정의
income_groups = {1: 94, 2: 191, 3: 216, 4: 268, 5: 458}

# 소득분위에 따른 평균 소득 금액에 +-10% 오차 내 랜덤 값 할당
np.random.seed(42)
df['월소득(만원)'] = np.around(df['소득구간'].apply(lambda x: np.random.uniform(0.9, 1.1) * income_groups[x]))

# 결과 확인
print(df.head())

## 1.2 소비 비율, 저축/투자 비율, 부채 상환 비율 생성


<img src = "https://drive.google.com/uc?id=1YxEs7r8aTcKmZNMP4SOZptYBdNTzCsVD" height = 400 width = 340>

**<신한 보통사람 금융생활 보고서 2022 p.94 참조>**

In [ ]:
# 각 소득분위에 따른 소비, 저축/투자, 부채상환 비율 저장

# 소비 비율
expenditure_ratios = np.array([0.585, 0.482, 0.454, 0.437, 0.41])
# 저축/투자 비율
savings_invest_ratios = np.array([0.234, 0.298, 0.319, 0.291,0.238])
# 부채 상환 비율
debt_ratios = np.array([0.064, 0.052, 0.06, 0.075, 0.063])

In [ ]:
# 비율이 모두 합쳐졌을 때 1보다 크지 않은 것을 확인인
df.loc[df['소비비율']+df['저축/투자비율']+df['부채상환비율'] >= 1]

## 1.3 저축/투자 비율, 연금저축(투자상품) 비율 생성과 그에 다른 금액 생성

<img src = "https://drive.google.com/uc?id=1D26bFtAtfZ44qp581zpcMYRnEjJ0ENpF" height = 400 width = 340>

**<신한 보통사람 금융생활 보고서 2022 p.22 참조>**

In [ ]:
# 적금청약 35%, 보험 37.8%, 투자상품 13.6%
#적금.청약 비율
Savings_ratios = 0.35
#연금저축(투자상품) 비율
Investment_ratios = 0.136

In [ ]:
np.random.seed(42)
total_savings_invest_ratio = df['저축/투자비율']

# 비율 초기화.
df['적금/청약비율'] = 0
df['연금저축(투자상품)비율'] = 0

# 비율에 맞게 랜덤 값을 할당
for i in range(1, 1+len(df)):
    # 총 지출 비율을 제외한 비율의 합
    total_ratio = Savings_ratios + Investment_ratios
    
    # +-10% 오차 범위 내에서 랜덤 값 할당
    savings_ratio = np.random.uniform(0.9, 1.1) * Savings_ratios * total_savings_invest_ratio[i] / total_ratio
    investment_ratio = np.random.uniform(0.9, 1.1) * Investment_ratios * total_savings_invest_ratio[i] / total_ratio
    
    # 할당한 랜덤 값으로 열 할당
    df.at[i, '적금/청약비율'] = savings_ratio
    df.at[i, '연금저축(투자상품)비율'] = investment_ratio

#소수점 셋째자리 수까지 표현
df[['적금/청약비율', '연금저축(투자상품)비율']] = \
    df[['적금/청약비율', '연금저축(투자상품)비율']].round(3)

df

In [ ]:
# 적금/청약 비율에 따른 적금/청약액, 연금저축(투자상품)비율에 따른 연금저축(투자상품)액 추가
df['적금/청약액'] = np.around(df['적금/청약비율'] * df['월소득(만원)']).astype(int)
df['연금저축(투자상품)액'] = np.around(df['연금저축(투자상품)비율'] * df['월소득(만원)']).astype(int)
df

## 1.4 소비 세부 카테고리 생성


<img src = "https://drive.google.com/uc?id=15aObtdWZKrorPqKQCjLxdndaMnic6SDq" height = 450 width = 350>

**<신한 보통사람 금융생활 보고서 2022 p.22>**

In [ ]:
#소비 세부 카테고리열 추가를 위해 기존 df 복사 후 소득구간에 따라 sorting
df_copy = df.copy()
df_copy = df_copy.sort_values(by = ["소득구간"])

In [ ]:
# 신한 보통사람 금융생활 보고서 2022 p.95 참조
# 소득 구간 별 각 소비 카테고리 지출액의 합(=총 지출액)
total_spend_1 = 17 + 6 + 6 + 6 + 20 + 10 + 9 + 6 + 8 + 11 + 6
total_spend_2 = 25 + 10 + 8 + 6 + 27 + 16 + 14 + 7 + 15 + 15 + 8
total_spend_3 = 27  + 12 + 8 + 8 + 26 + 17 + 14 + 9 + 18 + 12 + 8
total_spend_4 = 29 + 13 + 9 + 9 + 29 + 20 + 15 + 8 + 22 + 19 + 12
total_spend_5 = 47 + 22 + 14 + 13 + 38 + 29 + 27 + 15 + 33 + 32 + 19

In [ ]:
# 소득 분위별 카테코리 금액
section_1 = [ 17 , 6 ,  6 , 10 , 11 , 9  ]
section_2 = [ 15 , 10 ,  8 , 16 , 15 , 14  ]
section_3 = [ 27 , 12 ,  8 , 17 , 12 , 14  ]
section_4 = [ 29 , 13 ,  9 , 20 , 19 , 15  ]
section_5 = [ 47 , 22 ,  14 , 29 , 32 , 27 ]

In [ ]:
# data_frame to array
df_array = df.values.tolist()

In [ ]:
# 행 하나씩 불러와 각 소득구간의 section에 추가
for i, row in enumerate(df_array, start = 0) :
    if row[1] == 1 :
        df_array[i].extend(section_1)
    elif row[1] == 2 :
        df_array[i].extend(section_2)
    elif row[1] == 3 :
        df_array[i].extend(section_3)
    elif row[1] == 4 :
        df_array[i].extend(section_4)
    elif row[1] == 5 :
        df_array[i].extend(section_5)

In [ ]:
# 기존의 모든 컬럼을 total_cols에 저장
total_cols = df.columns.values.tolist()

#추가할 컬럼을 spend_col에 저장
spend_col = ["외식/카페 비율","주유/교통 비율", "통신비 비율", "항공/여행 비율", "문화/교육 비율", "쇼핑 비율"]

# columns 붙이기
total_cols.extend(spend_col)
total_cols

In [ ]:
# 소비 카테고리를 합친 최종 데이터프레임
concated_df = pd.DataFrame(data = df_array,
                           columns = total_cols)

In [ ]:
# 각 소비 카테고리의 값의 변화를 위해 -10% ~ +10% 사이의 랜덤값 추출
food_random = np.random.randint(-10,10,10000) * 0.01 
transport_random = np.random.randint(-10,10,10000) * 0.01
tele_random = np.random.randint(-10,10,10000) * 0.01
edu_random = np.random.randint(-10,10,10000) * 0.01
shop_random = np.random.randint(-10,10,10000) * 0.01
air_random = np.random.randint(-10,10,10000) * 0.01

# -10% ~ +10% 사이의 랜덤값 적용
concated_df["외식/카페 비율"] = concated_df["외식/카페 비율"]  + concated_df["외식/카페 비율"] * np.round(food_random, 1)
concated_df["주유/교통 비율"] = concated_df["주유/교통 비율"] + concated_df["주유/교통 비율"] * np.round(transport_random, 1)
concated_df["통신비 비율"] = concated_df["통신비 비율"] + concated_df["통신비 비율"] * np.round(tele_random, 1)
concated_df["문화/교육 비율"] = concated_df["문화/교육 비율"] + concated_df["문화/교육 비율"] * np.round(edu_random, 1)
concated_df["쇼핑 비율"] = concated_df["쇼핑 비율"] + concated_df["쇼핑 비율"] * np.round(shop_random, 1)
concated_df["항공/여행 비율"] = concated_df["항공/여행 비율"] +  concated_df["쇼핑 비율"] * np.round(shop_random, 1)

In [ ]:
# 소득구간 분리하여 각 변수에 할당
df_1 = concated_df[concated_df["소득구간"] == 1.0].copy()
df_2 = concated_df[concated_df["소득구간"] == 2.0].copy()
df_3 = concated_df[concated_df["소득구간"] == 3.0].copy()
df_4 = concated_df[concated_df["소득구간"] == 4.0].copy()
df_5 = concated_df[concated_df["소득구간"] == 5.0].copy()

# 소비카테고리 비율로 변환
df_1.iloc[:, 13:] = np.round(concated_df[concated_df["소득구간"] == 1].copy().iloc[:,13:].values / total_spend_1, 3)
df_2.iloc[:, 13:] = np.round(concated_df[concated_df["소득구간"] == 2].copy().iloc[:,13:].values / total_spend_2, 3)
df_3.iloc[:, 13:] = np.round(concated_df[concated_df["소득구간"] == 3].copy().iloc[:,13:].values / total_spend_3, 3)
df_4.iloc[:, 13:] = np.round(concated_df[concated_df["소득구간"] == 4].copy().iloc[:,13:].values / total_spend_4, 3)
df_5.iloc[:, 13:] = np.round(concated_df[concated_df["소득구간"] == 5].copy().iloc[:,13:].values / total_spend_5, 3)

In [ ]:
# 랜덤값 적용, 비율로 변환한 데이터 병합
user_df = pd.concat([df_1, df_2, df_3, df_4, df_5], axis = 0)

In [ ]:
# 덜 사용한 카테고리 2개 선정
np.random.seed(42)
less_cat_1 = np.random.randint(0, 6, 1000)

# 중복 방지
less_cat_2 = []
for i in less_cat_1 :
    rand_num = np.random.randint(0, 6)
    if rand_num != i :
        less_cat_2.append(rand_num)
    else : 
        while True :
            rand_num = np.random.randint(0, 6)
            if rand_num != i :
                less_cat_2.append(rand_num)
                break

# 선정된 2개의 카테고리 병합
less_cat = np.stack([less_cat_1, less_cat_2], 1)

In [ ]:
# 선정된 2개의 카테고리는 -5% 적용
n = 0
for cat_1, cat_2 in less_cat :
    less_cat_1 = spend_col[cat_1]
    less_cat_2 = spend_col[cat_2]

    user_df[less_cat_1][n] = user_df[less_cat_1][n] - 0.05
    user_df[less_cat_2][n] = user_df[less_cat_2][n] - 0.05
    n += 1

In [ ]:
# 더 사용한 카테고리 1번째
more_spend_1 = []
for i in less_cat :
    random_num = np.random.randint(0, 6)
    if random_num not in i :
        more_spend_1.append(random_num)
    else : 
        while True :
            random_num = np.random.randint(0, 6)
            if random_num not in i :
                more_spend_1.append(random_num)
                break

# 더 사용한 카테고리 2번째
more_spend_2 = []
for i, cat in enumerate(less_cat, start = 0)  :
    random_num = np.random.randint(0, 6)
    if random_num not in cat and random_num != more_spend_1[i] :
        more_spend_2.append(random_num)
    else : 
        while True :
            random_num = np.random.randint(0, 6)
            if random_num not in cat and random_num != more_spend_1[i]:
                more_spend_2.append(random_num)
                break
# 2개의 카테고리 병합
more_cat = np.stack([more_spend_1, more_spend_2], 1)

# 선택된 카테고리 +5% 적용하여 할당
n = 0
for cat_1, cat_2 in more_cat :
    more_cat_1 = spend_col[cat_1]
    more_cat_2 = spend_col[cat_2]

    user_df[more_cat_1][n] = user_df[more_cat_1][n] + 0.05
    user_df[more_cat_2][n] = user_df[more_cat_2][n] + 0.05
    n += 1

## 1.5 주택청약 가입, 연금 저축 가입 여부 생성

**연령대별 청약통장 가입 비중 참고**


<img src = "https://drive.google.com/uc?id=1amLVXNQUdaW4k9aUD7JfUJkwaM_Fxfnr" height = 300 width = 380>\\


**<연금 저축 가입의 경우 2021년 기준 20대 633만명, 20대 가입자 62만3천명 ->  62.3/633 = 9.84%로 역환산>**

In [ ]:
# 주택청약, 연금저축 가입 여부 열 추가
np.random.seed(42)
house_values = np.random.choice([0, 1], size=10000, p=[0.328, 0.672])
savings_values = np.random.choice([0, 1], size = 10000, p=[0.0984, 0.9016])
user_df['주택청약가입'] = house_values
user_df['연금저축가입'] = savings_values
user_df.columns

In [ ]:
# 정수형으로 변환
user_df[['연령', '소득구간', '월소득(만원)', '소비액', '저축/투자액','부채상환액', '적금/청약액', '연금저축(투자상품)액']] = user_df[['연령', '소득구간', '월소득(만원)', '소비액', '저축/투자액','부채상환액', '적금/청약액', '연금저축(투자상품)액']].astype(int)
user_df

In [ ]:
user_df.to_csv("0512_Final_User_Data.csv")

In [ ]:
df = user_df.copy()

# 2.군집화

*   사용자 맞춤 서비스 제공을 위해 유사한 유형의 사용자 간 군집화를 진행
*   kmeans 군집화 사용







In [ ]:
# 군집화 평가 metrics
from sklearn.metrics import  silhouette_score

def kmeans_modeling(df, n) : 
    # n : cluster 개수
    # df : 학습데이터

    # kmeans 군집화
    kmeans = KMeans(n_clusters = n, init = "k-means++", n_init = 10, max_iter =300, random_state = 42)

    # 군집화 수행
    kmeans.fit_predict(df)

    # 군집 결과 예측
    df["cluster"] = kmeans.labels_

    # 모든 데이터 평균 실루엣 개수값을 구함
    average_score = silhouette_score(df, df['cluster'])

    # 데이터 프레임과 평균 실루엣 개수를 반환
    return df, average_score


In [ ]:
"""
군집개수 선정 근거
최소 군집 개수 : 소득구간 개수를 기준 (5개)
최대 군집 개수 : n_cluster option을 주지 않았을 때 개수를 기준 (8개)
"""
max_values = []

for i in range(5, 9) : 
    cluster_df = df.copy()

    _, score = kmeans_modeling(cluster_df, i) # 실루엣 개수값
    max_values.append((i, score))

In [ ]:
# 군집 개수가 5일 때 가장 점수가 높은 것을 확인
max_values

In [ ]:
# 군집 5개로 군집화 진행
cluster_df, _=  kmeans_modeling(df, 5)

In [ ]:
cluster_df["cluster"] = cluster_df["cluster"]+1

In [ ]:
cluster_df["cluster"].value_counts()

## 2.1시각화

*   사용자 데이터 시각화 서비스 통해 한눈에 사용자의 금융 데이터 현황을 파악
*  시각화 데이터를 통해 계획적인 소비를 유도


In [ ]:
# 군집별 dataframe 저장
for i in range(1, 6) :
    globals()["cluster_{}".format(i)]  =  cluster_df[cluster_df["cluster"] == i]

# 군집 list에 저장
cluster_list = [cluster_df, cluster_1, cluster_2, cluster_3, cluster_4, cluster_5]

In [ ]:
# 소비 데이터 시각화
def to_boxplot_cat(df_list) :
    fig, axs = plt.subplots(figsize = (48, 8), nrows = 1, ncols = 6, squeeze = False)
    for i, cat in enumerate(df_list, start = 0) :
        a = sns.boxplot(cat.iloc[:,13 : 19], ax = axs[0][i])
        a.set_xticklabels(a.get_xticklabels(), rotation=45, ha='right')

In [ ]:
to_boxplot_cat(cluster_list)

In [ ]:
# 저축 데이터 시각화
def to_boxplot_save(df_list) :
    fig, axs = plt.subplots(figsize = (48, 8), nrows = 1, ncols = 6, squeeze = False)
    for i, cat in enumerate(df_list, start = 0) :
        a = sns.boxplot(cat.iloc[:,[3,4,5,9,10]], ax = axs[0][i])
        a.set_xticklabels(a.get_xticklabels(), rotation=45, ha='right')

In [ ]:
to_boxplot_save(cluster_list)

### 사용자와 군집 비교

* 사용자와 군집간 지출금액을 비교해 어떤 부분에서 과소비가 이루어졌는지 정보를 제공

In [ ]:
cluster_list[2].iloc[: , 13 : 19]

In [ ]:
# 사용자 1 
user_1 = pd.DataFrame(data = cluster_1.iloc[0])
user_1 = user_1.transpose()

In [ ]:
def user_cat_compare(user, cluster_list) :
    c_list = cluster_list.copy()
    cluster = user["cluster"]
    sns.barplot(user.iloc[:,13 : 19], color = "red", alpha = 0.5, label = "사용자")
    sns.barplot(c_list[int(cluster)].iloc[:,13 : 19], color = "blue", alpha = 0.5, label = "군집")
    plt.title("사용자와 군집 소비 비교")
    plt.legend()

def user_save_compare(user, cluster_list) :
    c_list = cluster_list.copy()
    cluster = user["cluster"]
    sns.barplot(user.iloc[:,[3,4,5,9,10]], color = "red", alpha = 0.5, label = "사용자")
    sns.barplot(c_list[int(cluster)].iloc[:,[3,4,5,9,10]], color = "blue", alpha = 0.5, label = "군집")
    plt.title("사용자와 군집 저축 비교")
    plt.legend()

In [ ]:
user_cat_compare(user_1, cluster_list)

In [ ]:
user_save_compare(user_1, cluster_list)

### 연령대 평균과 개인 사용자 비교

* 자신과 비슷한 연령대의 사용자가 어떤 카테고리에 돈을 사용하는지 정보를 시각화 하는 서비스를 제공


* 해당 서비스를 통해 좀 더 나은 제테크가 가능할 것으로 예상

In [ ]:
import plotly.express as px

In [ ]:
def compare_rader_chart(user, df) : 
    # df 평균
    mean_df = df.copy()
    mean_df = pd.DataFrame(mean_df.mean()).transpose().iloc[: , [6,7,8,11,12]].reset_index(drop = True)
    # user 데이터
    user = user.copy().iloc[: , [6,7,8,11,12]].reset_index(drop=True)
    # data concat
    concated_df = pd.concat([mean_df, user], axis = 1).transpose().reset_index()

    # user 구분
    concated_df["color"] = "total_user"
    concated_df.loc[5 :, "color"] = "user"

    return concated_df

In [ ]:
con_df = compare_rader_chart(user_1, cluster_df)

fig = px.line_polar(data_frame= con_df, r = 0, theta= "index", color="color", line_close=True)
fig.update_traces(fill='toself')


# 3.예상 가능한 최대 공제액 계산

*   [[참고] https://finda.co.kr/finance/calculator/yearendtax ](https://finda.co.kr/finance/calculator/yearendtax)
*   '연간 급여액의 25%를 초과하는 소비금액과 소비 수단 별 공제율의 평균값을 통해 산출한 값' 참고
*  공제율/ 공제한도, 급여별 총 한도액 참고 

In [ ]:
!sudo apt-get install -y fonts-nanum

In [ ]:
FONT_PATH = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=FONT_PATH, size=10).get_name()
fm.fontManager.addfont(FONT_PATH)
print(font_name)
plt.rcParams['font.family']=font_name
assert plt.rcParams['font.family'] == [font_name], "한글 폰트가 설정되지 않았습니다."

In [ ]:
def consumption_deduction(Salary, CreditCard_Expenses, CashReceipt_Expenses, DebitCard_Expenses):
    
    # 1. 연간 소비금액 계산
    # 연간 소비금액 = 총 소비액 - (연간 급여액 x 0.25)
    # 총 소비액  = 신용카드 사용액, 현금 영수증 사용액, 체크카드 사용액
    Total_Spending = CreditCard_Expenses + CashReceipt_Expenses + DebitCard_Expenses
    Yearly_Spending = abs(Total_Spending - (Salary * 0.25))

    # 2. 기본 공제 한도 계산
    if Salary <= 70000000:
        Basic_Deduction_Limit = 3000000
    else:
        Basic_Deduction_Limit = 2500000

    # 3. 추가 공제 한도 계산
    if Salary <= 70000000:
        Additional_Deduction_Limit = 3000000
    else:
        Additional_Deduction_Limit = 2500000

    # 4. 공제율 평균값 계산
    #공제율 평균값 = (신용카드 공제율 + 현금영수증 공제율 + 체크카드 공제율) / 3
    CreditCard_Deduction_Rate = 15
    CashReceipt_Deduction_Rate = 30
    DebitCard_Deduction_Rate = 30

    Average_Deduction_Rate = (CreditCard_Deduction_Rate + CashReceipt_Deduction_Rate + DebitCard_Deduction_Rate) / 3

    # 5. 예상 공제 금액 계산
    # 실제 공제 금액 = min(연간 소비금액, 기본 공제 한도 + 추가 공제 한도)
    Actual_Deduction_Amount = min(Yearly_Spending, Basic_Deduction_Limit + Additional_Deduction_Limit)
    Actual_Deduction_Amount = int(Actual_Deduction_Amount)

    return Actual_Deduction_Amount 

In [ ]:
# 예시 인자
Salary = 20000000
CreditCard_Expenses = 300000
CashReceipt_Expenses = 300000
DebitCard_Expenses = 300000

def format_amount(amount):
    if amount >= 10000:
        return f"{amount // 10000}만원"
    else:
        return f"{amount}원"

actual_deduction = consumption_deduction(Salary, CreditCard_Expenses, CashReceipt_Expenses, DebitCard_Expenses)

formatted_actual_deduction = format_amount(actual_deduction)

print("예상 공제 금액:", formatted_actual_deduction)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def calculate_probabilities(Salary, CreditCard_Expenses, CashReceipt_Expenses, DebitCard_Expenses):
    total_expenses = CreditCard_Expenses + CashReceipt_Expenses + DebitCard_Expenses

    salary_prob = Salary / (Salary + total_expenses)
    credit_card_expenses_prob = CreditCard_Expenses / (Salary + total_expenses)
    cash_receipt_expenses_prob = CashReceipt_Expenses / (Salary + total_expenses)
    debit_card_expenses_prob = DebitCard_Expenses / (Salary + total_expenses)

    return salary_prob, credit_card_expenses_prob, cash_receipt_expenses_prob, debit_card_expenses_prob

def print_probabilities(probabilities):
    labels = ['급여액', '신용카드', '현금 영수증', '체크카드']
    for label, prob in zip(labels, probabilities):
        print(label + '(%): {:.2f}'.format(prob))

def draw_pie_chart(probabilities):
    labels = probabilities.index
    plt.pie(probabilities, labels=labels, autopct='%1.1f%%')
    plt.axis('equal')
    plt.title('카드 및 현금 소비액')
    plt.show()

def analyze_financial_data(Salary, CreditCard_Expenses, CashReceipt_Expenses, DebitCard_Expenses):
    probabilities = calculate_probabilities(Salary, CreditCard_Expenses, CashReceipt_Expenses, DebitCard_Expenses)
    print_probabilities(probabilities)

    # 확률 값 데이터프레임 생성
    probabilities_df = pd.Series(probabilities,
                                 index=['급여액', '신용카드', '현금 영수증', '체크카드'])

    # 원 그래프 그리기
    draw_pie_chart(probabilities_df)

# 입력값
Salary = 3000000
CreditCard_Expenses = 500000
CashReceipt_Expenses = 300000
DebitCard_Expenses = 200000

# 데이터 분석 및 시각화
analyze_financial_data(Salary, CreditCard_Expenses, CashReceipt_Expenses, DebitCard_Expenses)

## 3.1시각화

### 고객ID 별 Radar Chart
* 시각화 : Radar Chart
* 함수 : plot_radar_chart(df, customer_id)
* 카테고리 : '저축/투자비율', '적금/청약비율', '연금저축(투자상품)비율', '소비비율',  '외식/카페 비율'

In [ ]:
## matplotlib - Radar Chart

import matplotlib.pyplot as plt
import numpy as np


def plot_radar_chart(df, customer_id):
    # 데이터 설정
    categories = ['저축/투자비율', '적금/청약비율', '연금저축(투자상품)비율', '소비비율',  '외식/카페 비율']
    values = df.loc[data['고객ID'] == customer_id, categories].values.flatten().tolist()
    print(values)

    # 데이터 개수 및 각도 계산
    num_vars = len(categories)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    values += values[:1]  # 첫 번째 값과 마지막 값을 연결하여 폐곡선 형태로 만듦
    angles += aplot_radar_chart(df, 5000)ngles[:1]

    # Radar chart 그리기
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'polar': True})

    ax.fill(angles, values, color='skyblue', alpha=0.5)  # 폐곡선 색상 및 투명도 설정
    ax.plot(angles, values, color='skyblue', linewidth=1)  # 폐곡선 그리기

    ax.set_xticks(angles[:-1])  # 각 카테고리 각도 설정
    ax.set_xticklabels(categories)  # 각 카테고리 레이블 설정
    ax.set_yticklabels([])  # y축 레이블 숨기기

    ax.spines['polar'].set_visible(False)  # 테두리 숨기기

    ax.set_xticklabels(categories, fontproperties=fontprop)  # 한글 폰트 적용

    plt.show()

In [ ]:
plot_radar_chart(df, 5000)

### 고객ID 별 소비 카테고리 비율

* 시각화 : Bar Chart
*  함수 : customer_spending_category_bar_chart(customer_id, df)
* 카테고리 : '외식/카페 비율', '주유/교통 비율', '통신비 비율', '항공/여행 비율', '문화/교육 비율', '쇼핑 비율'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import pandas as pd

def customer_spending_category_bar_chart(customer_id, df):
    
    # 카테고리 및 비율 컬럼 선택
    categories = ['외식/카페 비율', '주유/교통 비율', '통신비 비율', '항공/여행 비율', '문화/교육 비율', '쇼핑 비율']
    values = df.loc[df['고객ID'] == customer_id, categories].values.flatten().tolist()
    print(values)
    
    # 데이터프레임 생성
    df_1 = pd.DataFrame({'카테고리': categories, '비율': values})
    
    # 가로 막대 그래프 그리기
    sns.barplot(x='비율', y='카테고리', data=df_1, color='skyblue')
    
    # 그래프에 제목 추가
    plt.title(f"(고객ID: {customer_id})")
    
    # 그래프 보여주기
    plt.show()


In [ ]:
customer_spending_category_bar_chart(3000, df)

### 군집 별 소비 카테고리 시각화

*  KMeans 알고리즘을 통해 군집화
*  시각화 : Boxplot
*  함수 : spending_category_boxplot(df)
*  카테고리 : '외식/카페 비율', '주유/교통 비율', '통신비 비율', '항공/여행 비율', '문화/교육 비율', '쇼핑 비율'

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

kmeans = KMeans(init = 'k-means++', max_iter = 300 ,random_state=1)

cluster = kmeans.fit(df)

# 군집 할당 결과 확인
df['cluster'] = kmeans.labels_

#df['cluster'].value_counts()

In [ ]:
# cluter 정렬하기 
unique_clusters = df['cluster'].unique()
sorted_clusters = sorted(unique_clusters)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def spending_category_boxplot(df):
    selected_columns = ['외식/카페 비율', '주유/교통 비율', '통신비 비율', '항공/여행 비율', '문화/교육 비율', '쇼핑 비율']

    for cluster_value in sorted_clusters:
        grouped_data = df[df['cluster'] == cluster_value][selected_columns]
        
        sns.boxplot(data=grouped_data)
        plt.xticks(rotation=45)
        plt.title(f"Cluster {cluster_value}")
        plt.show()


In [ ]:
# 데이터프레임을 함수에 전달하여 그룹화된 boxplot 그리기
spending_category_boxplot(df)

### 군집별 시각화

*   KMeans 알고리즘을 통해 군집화
*   시각화 : Boxplot
*  함수 : grouped_boxplot(df)
* 카테고리 : '소비비율', '저축/투자비율', '부채상환비율', '적금/청약비율', '연금저축(투자상품)비율'


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def grouped_boxplot(df):
    selected_columns = ['소비비율', '저축/투자비율', '부채상환비율', '적금/청약비율', '연금저축(투자상품)비율']

    for cluster_value in sorted_clusters:
        grouped_data = df[df['cluster'] == cluster_value][selected_columns]

        sns.boxplot(data=grouped_data)
        plt.xticks(rotation=45)
        plt.title(f"Cluster {cluster_value}")
        plt.show()

### 고객ID vs 군집 소비 카테고리 비율



*   고객ID 별 소비 카터고리 비율 시각화
*   해당 고객이 속하는 군집 소비 카테고리 비율 시각화 
* 시각화 : Bar Chart
* 함수 : plot_comparison(df, customer_id)
*  카테고리 : '외식/카페 비율', '주유/교통 비율', '통신비 비율', '항공/여행 비율', '문화/교육 비율', '쇼핑 비율'

In [ ]:
import matplotlib.pyplot as plt

def plot_comparison(df, customer_id):
    # 입력한 고객 ID의 행에서 'cluster' 값을 저장
    cluster = df.loc[data['고객ID'] == customer_id, 'cluster'].values[0]

    # 고객 ID에 해당하는 행의 소비 카테고리 칼럼들을 가져옴
    customer_ratios = df.loc[data['고객ID'] == customer_id, ['외식/카페 비율', '주유/교통 비율', '통신비 비율', '항공/여행 비율', '문화/교육 비율', '쇼핑 비율']].values[0]

    # 'cluster' 값에 해당하는 행들의 소비 카테고리 칼럼들을 가져옴
    cluster_ratios = df.loc[data['cluster'] == cluster, ['외식/카페 비율', '주유/교통 비율', '통신비 비율', '항공/여행 비율', '문화/교육 비율', '쇼핑 비율']].mean()

    # 입력한 고객 ID의 그래프 그리기
    plt.plot(range(6), customer_ratios, label='고객')
    # 군집화된 그래프 그리기
    plt.plot(range(6), cluster_ratios, label='해당하는 Cluster')
    
    # 범례 추가
    plt.legend()
    
    # 그래프 설정
    plt.xticks(range(6), ['외식/카페', '주유/교통', '통신비', '항공/여행', '문화/교육', '쇼핑'])
    #plt.xlabel('비율 카테고리')
    plt.ylabel('비율')
    plt.title('소비 카테고리 비교 : Customer vs Cluster')
    
    # 그래프 출력
    plt.show()

In [ ]:
plot_comparison(df,1)

# 4.적금 추천 코드



*   상품 가입 정보 임시 데이터 생성
*   추천 시스템에서 추천할 상품의 우선도를 적용시킬 데이터
전체 고객 데이터에서 '주택청약가입', '연금저축가입' 데이터를 이용해 각 고객의 청약 상품 및 연금 상품 가입 여부 판단
추가로 전체 고객들이 청약과 연금 상품을 제외한 적금 상품에 모두 가입해있다는 전제를 두어 데이터 생성



In [ ]:
df2 = pd.read_csv('Final_User_Data.csv', index_col=0)
df2.reset_index(names='고객ID', inplace=True)
df2.loc[:, '고객ID'] = df2['고객ID']+1  # 각 상품에 가입한 고객들을 구분하기 위해 고객ID 추가

pro = pd.read_csv('prod_info2.csv', encoding='cp949')

In [ ]:
# 청약 가입자들 대상으로 청약 상품 random 선택
data1 = pd.DataFrame()
pro1 = list(pro.loc[pro['prod_name'].str.contains('주택청약')]['prod_name'].values)
pro1_cust = df2.loc[df2['주택청약가입']==1]
data1['고객ID'] = pro1_cust['고객ID']
data1['가입 상품'] = [np.random.choice(pro1) for _ in range(len(pro1_cust))]
data1

In [ ]:
# 전체 고객 대상으로 청약을 제외한 나머지 상품들 중 random 선택
data2 = pd.DataFrame()
pro2 = list(pro.loc[pro['prod_name'].str.contains('주택청약')==False]['prod_name'].values)
data2['고객ID'] = df2['고객ID']
data2['가입 상품'] = [np.random.choice(pro2) for _ in range(len(df2))]
data2

In [ ]:
# 연금 가입자들 대상으로 연금 상품 random 선택
data3 = pd.DataFrame()
pro3 = pd.read_excel('연금저축 상품 리스트.xlsx')
pro3 = list(pro3['상품 이름'].values)
pro3_cust = df2.loc[df2['연금저축가입']==1]
data3['고객ID'] = pro3_cust['고객ID']
data3['가입 상품'] = [np.random.choice(pro3) for _ in range(len(pro3_cust))]
data3

In [ ]:
# 앞에서 만든 3개의 데이터를 합친 후 고객ID 기준으로 정렬해 최종 데이터 생성
prod_data = pd.concat([data1, data2, data3])
prod_data.sort_values(by='고객ID', inplace=True)
prod_data

## 4.1적금상품 추천코드


### 적금 상품 추천 함수
 

*   함수에 상품을 추천 받을 사용자의 데이터 입력
*   사용자의 군집을 예측
* 예측한 군집에 속해있는 다른 고객들의 데이터를 불러옴
* 해당 군집에서 사용자의 데이터와 일치율이 높은 고객들이 이용중인 상품들 중 군집 내에서 많은 고객들이 가입한 상품들을 사용자에게 추천

In [ ]:
def prod_recom(user):  # 사용 시 추천 받을 고객의 데이터 입력
    df = pd.read_csv('Final_User_Data.csv', index_col=0) 

    # 전체 고객 데이터 기반으로 군집화
    kmeans = KMeans(random_state=20)
    kmeans.fit(df)

    # 데이터에 군집 정보 추가
    df['cluster'] = kmeans.labels_

    # 고객별 가입 상품에 대한 데이터를 활용하기 위해 고객을 구별할 값 필요 > 고객 ID 추가
    df.reset_index(names='고객ID', inplace=True)
    df.loc[:, '고객ID'] = df['고객ID']+1

    # 사용자의 군집 예측
    cluster = kmeans.predict([user])[0]

    # 사용자의 군집에 해당되는 데이터들만 추출
    df_tmp = df.loc[df['cluster']==cluster]
    
    # 상품 가입 정보 데이터 불러오기
    prod = pd.read_csv('prod_data.csv', encoding='cp949')

    result = {}
    prods = prod.loc[prod['고객ID'].isin(df.loc[df['cluster']==cluster]['고객ID'])]

    # 가중치 1 : 상품 가입자수(가입자가 많은 상품을 더 높은 순위로 추천) > 순위를 가중치 1로 사용
    # 청약상품, 연금상품, 나머지 상품별로 각 상품별 가입자수 추출(pr1: 청약, pr2: 연금, pr3: 나머지) 
        # > 상품 종류별로 따로 순위를 산출하기 위해 구분해서 계산(상품 구별을 안하면 청약 상품이 무조건 상위를 차지해서 구별한 것)
    pr1 = pd.DataFrame(prods.loc[prods['가입 상품'].str.contains('주택청약')]['가입 상품'].value_counts())
    pr2 = pd.DataFrame(prods.loc[prods['가입 상품'].str.contains('연금')]['가입 상품'].value_counts())
    pr3 = pd.DataFrame(prods.loc[(prods['가입 상품'].str.contains('주택청약')==False) & 
                                  (prods['가입 상품'].str.contains('연금')==False)]['가입 상품'].value_counts())
    pr1.columns=['count']
    pr2.columns=['count']
    pr3.columns=['count']
    # 가입 상품=상품명이고 value_counts의 경우 내림차순으로 자동 정렬해서 결과를 반환해주기 때문에 
        # index에서 상품명을 제거하면 새로 부여되는 index가 순위가 됨
    pr1 = pr1.reset_index(names='가입 상품').reset_index(names=['weight1'])
    pr2 = pr2.reset_index(names='가입 상품').reset_index(names=['weight1'])
    pr3 = pr3.reset_index(names='가입 상품').reset_index(names=['weight1'])
    
    pr4 = pd.concat([pr1, pr2, pr3])

    # 가중치 2 : 사용자의 정보와 비슷한 정도(사용자와 정보가 더 가까운 고객이 가입한 상품일수록 더 높은 순위로 추천)
    # 모든 데이터에 사용자의 데이터 값을 빼준 값들 중 월소득과 각종 소비 비율들만 비교에 사용
    df_dif = (df_tmp-user)[['월소득(만원)', '소비비율', '저축/투자비율', '부채상환비율', '적금/청약비율', '연금저축(투자상품)비율']]
    df_dif['고객ID'] = df_tmp['고객ID']

    # 값의 범위가 비슷한 column들을 모아 평균을 구하고 절댓값으로 변환
        # 절댓값 > +- 관계 없이 0에 가까울수록 사용자와의 정보와 비슷한 것이기 때문에 이후에 정렬할 때 0을 상위에 놓기 편하게 데이터 변환
    # 소비 평균의 경우 그냥 사용할 경우 값의 차이가 너무 작아 *100를 해준 것
    df_dif['소비평균차이(abs)'] = abs((df_dif['소비비율']+df_dif['저축/투자비율']+df_dif['부채상환비율']+df_dif['적금/청약비율']+df_dif['연금저축(투자상품)비율']))*100/5
    df_dif['월소득(만원)'] = abs(df_dif.loc[:, '월소득(만원)'])

    # 단순 합(소득+소비) > 월소득의 차이는 기본적으로 정수값이고 크면 100단위까지 커지기 때문에 소비 차이는 고려되지 않을 확률이 큼
    # 단순 곱(소득*소비) > 두 값 중 하나라도 0이면 무조건 최종값이 0이 나오기 때문에 가중치로 활용하기 적합하지 않음
    # >> 합과 곱의 값을 모두 활용(소득+소비+소득*소비) > 두 값의 합이 비슷해도 곱한 값으로 인해 차이를 발생시킬 수 있을 것이라 생각
    df_dif['weight2'] = df_dif['월소득(만원)']+df_dif['소비평균차이(abs)']+df_dif['월소득(만원)']*df_dif['소비평균차이(abs)']

    # 상품 가입 데이터를 기준으로 계산해둔 가중치값 붙이기 : 기준이 되는 dataframe에 중복되는 값들(고객ID, 가입상품)에도 값을 다 붙여주기 위해 left 사용
    result = prods.merge(df_dif, how='left')  # 고객 ID 기준
    result = result.merge(pr4, how='left')    # 가입 상품 기준

    # 최종 가중치는 가중치1+가중치2
    result['weight'] = result['weight1']+result['weight2']

    # 사용자가 청약 가입자인 경우 최종 결과에서 청약 상품 제외
    if user['주택청약가입']==1:
        result.drop(result.loc[result['가입 상품'].str.contains('주택청약')].index, inplace=True)
    # 사용자가 연금 가입자인 경우 최종 결과에서 연금 상품 제외
    if user['연금저축가입']==1:
        result.drop(result.loc[result['가입 상품'].str.contains('연금')].index, inplace=True)

    # 최종 결과는 상품별로 가중치의 평균을 구해 오름차순으로 정렬 후(0에 가까울수록 상위) 상위 10개의 상품명 반환
    return list(result.groupby('가입 상품')['weight'].mean().sort_values().head(10).index)

[ 결과 예시 ]

In [ ]:
# 청약가입=1, 연금가입=1
prod_recom(df.loc[0])

In [ ]:
# 청약가입=1, 연금가입=1
prod_recom(df.loc[3])

In [ ]:
# 청약가입=0, 연금가입=0
prod_recom(df.loc[6])

In [ ]:
# 청약=1, 연금=0
prod_recom(df.loc[23])

## 4.2적금금액 예측 코드

### 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='NanumBarunGothic')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
# 군집화 평가 metrics
from sklearn.metrics import  silhouette_score

### 군집화

* 비슷한 사용자의 적금/청양 및 연금저축 금액을 추천해 주기 위해 군집화 진행

In [ ]:
def kmeans_modeling(df, n) : 
    # n : cluster 개수
    # df : 학습데이터

    # kmeans 군집화
    kmeans = KMeans(n_clusters = n, init = "k-means++", n_init = 10, max_iter =300, random_state = 42)

    # 군집화 수행
    kmeans.fit_predict(df)

    # 군집 결과 예측
    df["cluster"] = kmeans.labels_

    # 모든 데이터 평균 실루엣 개수값을 구함
    average_score = silhouette_score(df, df['cluster'])

    # 데이터 프레임과 평균 실루엣 개수를 반환
    return df, average_score


In [ ]:
"""
군집개수 선정 근거
최소 군집 개수 : 소득구간 개수를 기준 (5개)
최대 군집 개수 : n_cluster option을 주지 않았을 때 개수를 기준 (8개)
"""
max_values = []

for i in range(5, 9) : 
    cluster_df = df.copy()

    _, score = kmeans_modeling(cluster_df, i) # 실루엣 개수값
    max_values.append((i, score))

In [ ]:
# 군집 개수가 5일 때 가장 점수가 높은 것을 확인
max_values

In [ ]:
# 군집 5개로 군집화 진행
cluster_df, _=  kmeans_modeling(df, 5)

In [ ]:
cluster_df["cluster"] = cluster_df["cluster"]+1

In [ ]:
cluster_df["cluster"].value_counts()

### 딥러닝



* 딥러닝 기술을 이용해 최적의 적금 및 연금 저축 금액을 추천하는 서비스 제공



In [ ]:
# 데이터 정규화
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# 군집화 나누기
cluster_df, x_test = train_test_split(cluster_df, test_size= 0.1, shuffle= True, random_state = 42)

In [ ]:
# # 군집별 dataframe 저장
for i in range(1, 6) :
    globals()["cluster_{}".format(i)]  =  cluster_df[cluster_df["cluster"] == i]

# 군집 list에 저장
cluster_list = [cluster_1, cluster_2, cluster_3, cluster_4, cluster_5]

### 모델링



* 군집별 데이터 학습을 통해 최적의 적금 및 연금 저축액을 추천



In [ ]:
# 예적금 모델
def deepLearnModeling_save(df) :
    x = df.drop(["적금/청약비율", "적금/청약액"], axis = 1)
    y = df.loc[:, ["적금/청약액"]]

    # 데이터 정규화
    scaler = MinMaxScaler()
    scaler.fit(x)
    x = scaler.transform(x)

    keras.backend.clear_session()

    # 딥러닝 레이어
    input_layer = keras.layers.Input(shape=(x.shape[1],))
    h1 =  keras.layers.Activation(activation = "relu")(input_layer)
    h1 =  keras.layers.Dense(32)(h1)
    h1 =  keras.layers.BatchNormalization()(h1)
    h1 =  keras.layers.Activation(activation = "relu")(h1)
    h1 =  keras.layers.Dense(64)(h1)
    h1 =  keras.layers.BatchNormalization()(h1)
    h1 =  keras.layers.Activation(activation = "relu")(h1)
    h1 =  keras.layers.Dense(32)(h1)
    h1 =  keras.layers.BatchNormalization()(h1)
    h1 =  keras.layers.Activation(activation = "relu")(h1)
    output_layer = keras.layers.Dense(1)(h1)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss='mse', optimizer='adam')

    # earlystopping
    es = EarlyStopping(monitor = "val_loss",
                    mode = "min",
                    patience = 10,
                    verbose = 1,
                    restore_best_weights = True)

    model.fit(x, y,  callbacks = [es], validation_split = 0.1, verbose = 1, epochs = 100)
    
    return model

# 연금 모델
def deepLearnModeling_invest (df) :
    x = df.drop(["연금저축(투자상품)비율","연금저축(투자상품)액"], axis = 1)
    y = df.loc[:, ["연금저축(투자상품)액"]]

    # 데이터 정규화
    scaler = MinMaxScaler()
    scaler.fit(x)
    x = scaler.transform(x)

    keras.backend.clear_session()

    # 딥러닝 레이어
    input_layer = keras.layers.Input(shape=(x.shape[1],))
    h1 =  keras.layers.Activation(activation = "relu")(input_layer)
    h1 =  keras.layers.Dense(32)(h1)
    h1 =  keras.layers.BatchNormalization()(h1)
    h1 =  keras.layers.Activation(activation = "relu")(h1)
    h1 =  keras.layers.Dense(64)(h1)
    h1 =  keras.layers.BatchNormalization()(h1)
    h1 =  keras.layers.Activation(activation = "relu")(h1)
    h1 =  keras.layers.Dense(32)(h1)
    h1 =  keras.layers.BatchNormalization()(h1)
    h1 =  keras.layers.Activation(activation = "relu")(h1)
    output_layer = keras.layers.Dense(1)(h1)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss='mse', optimizer='adam')

    # earlystopping
    es = EarlyStopping(monitor = "val_loss",
                    mode = "min",
                    patience = 10,
                    verbose = 1,
                    restore_best_weights = True)

    model.fit(x, y, validation_split = 0.1, callbacks = [es], verbose = 1, epochs = 100)
    
    return model

In [ ]:
# 예적금 모델링
for i, df in enumerate(cluster_list, start =1) :
    globals()["save_moedel_{}".format(i)] = deepLearnModeling_save(df.iloc[:, 2:19])

In [ ]:
# 연금 모델링
for i, df in enumerate(cluster_list, start =1) :
    globals()["invest_moedel_{}".format(i)] = deepLearnModeling_invest(df.iloc[:, 2:19])

### 금액 예측



> 사용자 데이터가 들어오면 맞춤 모델을 통해 저축금액 추천



In [ ]:
# 사용자 1 
user_1 = pd.DataFrame(data = x_test.iloc[0])
user_1 = user_1.transpose()
user_1

In [ ]:
user = user_1.iloc[:,2:19].copy()

#test 데이터 스케일링
scaler = MinMaxScaler()
scaler.fit(cluster_5.iloc[:,2:19].drop(["적금/청약비율", "적금/청약액"], axis = 1))
x_scaled = scaler.transform(user.drop(["적금/청약비율", "적금/청약액"], axis = 1))

In [ ]:
#실제 적금/청약액 54.0 예측 적금/청약액 52.22
save_moedel_5.predict(x_scaled)

In [ ]:
scaler_2 = MinMaxScaler()
scaler_2.fit(cluster_5.iloc[:,2:19].drop(["연금저축(투자상품)비율", "연금저축(투자상품)액"], axis = 1))
x_scaled_2 = scaler_2.transform(user.drop(["연금저축(투자상품)비율", "연금저축(투자상품)액"], axis = 1))

In [ ]:
# 실제 연금 저축액 21 예측 연금/저축액  25.3
invest_moedel_5.predict(x_scaled_2)

# 5.카드 추천 코드
* 소비율이 25%이상일 경우 체크카드 추천, 소비율이 25% 이하일 경우 신용카드 추천 / 카드는 상위 1개 최다소비 카테고리에 맞춰서 추천

### 사용 데이터

* Final_User_Data.csv : 전체 고객 데이터
* card_info.csv : 카드 정보 데이터 (필요 데이터만 뽑은 파일로 변경 시 card_info2.csv)

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

### 카드 상품 추천 함수

* 함수에 상품을 추천 받을 사용자의 데이터 입력
* 사용자의 군집을 예측
* 예측한 군집에 속해있는 다른 고객들의 데이터를 불러옴 
* 전체 데이터와 군집 데이터의 각 소비 카테고리별 소비 비율 평균을 구함
* 아래 순서로 추천할 카드 카테고리 선정


    1.   전체 평균과 군집 평균을 비교
    2.   (전체<군집) 인 카테고리에 대해 사용자의 데이터와 군집 평균을 비교
    3. (군집<사용자) 인 카테고리의 카드 추천
    4. 3번의 결과가 없을 시 2번의 결과에 맞는 카테고리의 카드 추천


---


* 카드 추천 결과는 (군집 vs 사용자) 또는 (전체 vs 군집) 각 카테고리별 차이를 계산해 소비 정도가 더 큰 카테고리일수록 더 상위에 배치
<br>
<br>
<b>[ 추가 ]</b>
- 추후에 각 고객별로 1월부터 현재까지의 소비액 또는 전체 소득 대비 소비율에 대한 데이터가 추가된다면 소비율(25%:소득공제 기준%)을 기준으로 25% 이상일 경우 체크카드, 25% 이하일 경우 신용카드 추천 가능

In [ ]:
def card_recom(user):  # 사용 시 추천 받을 고객의 데이터 입력

    # 전체 고객 데이터 기반으로 군집화
    kmeans = KMeans(random_state=20)
    kmeans.fit(df)

    # 데이터에 군집 정보 추가
    df['cluster'] = kmeans.labels_

    # 사용자의 군집 예측
    cluster = kmeans.predict([user])[0]

    # 사용자의 군집에 해당되는 데이터들만 추출
    df_tmp = df.loc[df['cluster']==cluster]

    # 카드 추천에 이용할 column들
    cate = ['외식/카페 비율', '주유/교통 비율', '통신비 비율', '항공/여행 비율', '문화/교육 비율', '쇼핑 비율']
    tot_m = df[cate].mean()     # 전체 고객들의 카테고리별 평균 소비 비율
    c_m = df_tmp[cate].mean()   # 사용자의 군집에 속하는 고객들의 카테고리별 평균 소비 비율

    # 카드 정보 불러오기
    card = pd.read_csv('card_info.csv', encoding='cp949')
    # 기존 카드 정보 데이터에서 일부 카테고리 제거 > 크롤링할 때 해당 카테고리들 제외하고 크롤링했다면 불필요한 과정
    card.drop(card.loc[card['cate'].isin(['생활', '공공/단체', '온라인쇼핑', '사업자'])].index, inplace=True)  

    # 카드 정보 불러오기2 > 사용할 카드 종류, 카테고리 정보만 있는 데이터
    # card = pd.read_csv('card_info2.csv', encoding='cp949')

    result = []      # 기본적으로 반환할 결과
    result_tmp = []  # result의 값이 없을 때 반환할 결과

    for i in range(6):  # 카테고리 수만큼 반복
        # 사용자가 해당되는 군집의 평균이 전체 평균보다 높고 유저 평균이 군집 평균보다 높은 카테고리의 카드 추천
        if tot_m[i]<c_m[i] and user[cate[i]]>c_m[i]:    # (전체<군집) & (군집<사용자)
            result += list(card.loc[card['cate']==cate[i].split(' ')[0]]['card_name'].values)
        elif tot_m[i]<c_m[i]:  # (전체<군집) > 위의 코드 실행 시 추천 카드 결과가 없는 상황을 방지하기 위한 코드
            result_tmp += list(card.loc[card['cate']==cate[i].split(' ')[0]]['card_name'].values)

    result_df = card.loc[card['card_name'].isin(result if len(result)>0 else result_tmp)]  # result 결과값이 없으면 tmp 결과 반환

    # 가중치 추가
    # 군집 평균과 비교해 사용자의 지출 정도가 더 큰 카테고리를 우선적으로 출력 > (사용자-군집)의 값이 클수록 상위
        # tmp 결과를 반환하는 경우 (군집-전체)의 값이 클수록 상위
    weight = [i.split(' ')[0] for i in (user[cate]-c_m if len(result)>0 else c_m-tot_m).sort_values(ascending=False).index]
    # sort_values의 결과는 index: 카테고리 비율, value: 평균 차이 > 인덱스 자체는 '외식/카페 비율' 등 > split으로 '비율' 제거
    # 최종 weight값은 평균 차이가 큰 카테고리 이름 순서
    
    # 카테고리별 사용 순위 추가(weight 결과가 카테고리 순서이므로 weight의 index=사용 순위)
    dic = {}
    for i in range(len(weight)):
        dic[weight[i]] = i
        
    # 각 카테고리별로 추가한 순위를 weight column으로 붙여준 뒤 weight 기준 오름차순으로 정렬(높은 순위에 있었던 카테고리가 상위)
    result_df['weight'] = result_df['cate'].map(dic) 
    result_df = result_df.sort_values(['weight', 'card_type'])

    # + 추가할 부분(조건문 추가 필요)
    # 신용카드만 반환할 경우
    # result_df = result_df.loc[result_df['card_type']=='신용카드']
    # 체크카드만 반환할 경우
    # result_df = result_df.loc[result_df['card_type']=='체크카드']

    return result_df.drop('weight', axis=1).reset_index(drop=True)

[ 결과 예시 ]

In [ ]:
card_recom(df.loc[1])